## 原始成績資料

In [1]:
# 修過的課
import bs4
import requests
import json
import sys
doneCourse = []
req = requests.Session()  # Seesion 儲存傳給伺服器的(對話)cookies
# if you wanna test this code, please input your account and password

# ==================== #
# 帳號密碼
account = 'a1085508'
password = '*********'
# ==================== #

req.post('https://aca.nuk.edu.tw/Student2/Menu1.asp',{'Account':account,'Password':password})
# Classno is your student ID
# 爬修過的課
r = req.post('https://aca.nuk.edu.tw/Student2/SO/ScoreQuery.asp',data={'Classno':''})
r.encoding = 'big5'
soup = bs4.BeautifulSoup(r.text,'html.parser')
soup = soup.select('table[border="1"] tr[align="center"] td')
for i in range(int(len(soup)/7)):
    compulsory = (soup[i*7+3].text == "必修")
    doneCourse.append([soup[i*7].text,soup[i*7+1].text,soup[i*7+2].text,soup[i*7+5].text,compulsory])
    
# 印出 done Course
print(json.dumps(doneCourse, ensure_ascii=False).encode("utf8",errors='ignore').decode("utf8",errors='ignore'))

[["AMA500", "基礎微積分", "2", "73", false], ["IFDA509", "史丹佛設計思考與科技創新", "2", "93", false], ["CSA001", "計算機概論", "2", "90", true], ["CSA011", "離散數學", "3", "90", true], ["CSA02A", "微積分(一)", "3", "87", true], ["CSA07A", "程式設計與實習（一）", "2", "97", true], ["GR5403", "中文(上)", "2", "95", true], ["GRA1B0", "英語會話與閱讀(上)：中階班", "2", "88", true], ["GRF109", "服務學習培養(創新學院 A)", "0", "95", true], ["GRS55A", "體育(一)-資工系", "0", "92", true], ["IFDA525", "數據分析應用概論", "0.5", "90", false], ["IFDA526", "大數據分析業界實務分享", "0.2", "90", false], ["IFDA534", "南科人工智慧發展現況與未來展望", "0.3", "90", false], ["LIB069", "職場英文聽讀訓練", "2", "93", false], ["SOB95A", "大學入門(工學院)", "1", "88", false], ["CSA02B", "微積分(二)", "3", "85", true], ["CSA041", "線性代數", "3", "85", true], ["CSA051", "數位系統導論", "3", "100", true], ["CSA07B", "程式設計與實習（二）", "2", "94", true], ["GR5302", "中文(下)", "2", "80", true], ["GRA0BH", "英語會話與閱讀(下)：中階班", "2", "95", true], ["GRA554", "服務學習培養(資訊工程學系B)", "0", "92", true], ["GRS55B", "體育(二)-資工系", "0", "93", true], ["CCC544", "科技與社會"

## 學生系所代碼

In [2]:
# 系所代碼處理
jsonObj = open('class_code.json','r', encoding='utf-8')
class_code = json.load(jsonObj)

course_code = {}
for code in class_code:
    course_code[code['text']] = code['code']

In [3]:
# 取得學生年度、系所、系所代碼
soup = bs4.BeautifulSoup(r.text,'html.parser')
student_aca = soup.find_all('td')[1].text[3:]
student_acayear = int(account[1:4])
student_aca_code = course_code[student_aca]

print(student_acayear, student_aca, student_aca_code)

108 資訊工程學系 CS


## 學生畢業門檻

In [4]:
r = req.post('https://aca.nuk.edu.tw/Graduate/GraduateData/QueryData.asp',
             data={'Classno': account.upper(), 'Pclass': account[0].upper(), 'Sclass': student_aca_code.lower(), 'Gclass': 999, 'Yclass': account.upper(), 'Year': int(account[1:4])})
r.encoding = 'big5'
soup = bs4.BeautifulSoup(r.text,'html.parser')
graduate_condition = soup.find_all('td')[4:18]

student_graduate_info = {}
for i in range(0, 14, 2):
    student_graduate_info[graduate_condition[i].text.replace('\u3000', '')] = graduate_condition[i+1].text

with open('StudentGraduateInfo.json','w',encoding='utf8') as f:
    f.write(json.dumps(student_graduate_info, ensure_ascii=False).encode("utf8",errors='ignore').decode("utf8",errors='ignore'))

## DataFrame 分析

In [5]:
import pandas as pd

In [6]:
df_doneCourse = pd.DataFrame(doneCourse, columns=['id', 'name', 'credit', 'score', 'category'])
df_doneCourse.head()

,id,name,credit,score,category
0,AMA500,基礎微積分,2,73,False
1,IFDA509,史丹佛設計思考與科技創新,2,93,False
2,CSA001,計算機概論,2,90,True
3,CSA011,離散數學,3,90,True
4,CSA02A,微積分(一),3,87,True


## Category 分類處理

In [7]:
# A1 系上必修 : 
# A2 系上選修 or 跨院 : 
# AC 中文
# AE 英文
# A3 共同必修 : 'GR'

# B1 核心通識 思維方法 : 'CCI1'
# B2 核心通識 美學素養 : 'CCI2'
# B3 核心通識 公民素養 : 'CCO3'
# B4 核心通識 文化素養 : 'CCO4'
# B5 核心通識 科學素養 : 'CCC5'
# B6 核心通識 倫理素養 : 'CCC6'

# C1 博雅通識 人文科學 : 'LI'
# C2 博雅通識 社會科學 : 'SO'
# C3 博雅通識 自然科學 : 'SC'

In [8]:
for course in doneCourse:
    course_id = course[0]
    if course_id[0:2] == 'CC':
        if course_id[0:4] == 'CCI1':
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'B1'
        elif course_id[0:4] == 'CCI2':
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'B2'
        elif course_id[0:4] == 'CCO3':
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'B3'
        elif course_id[0:4] == 'CCO4':
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'B4'
        elif course_id[0:4] == 'CCC5':
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'B5'
        elif course_id[0:4] == 'CCC6':
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'B6'
    elif course_id[0:2] == 'GR':
        if course_id[2:].isnumeric():
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'AC'
        elif course_id[-2].isalpha():
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'AE'
        else:
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'A3'
    elif course_id[0:2] == 'LI':
        df_doneCourse.loc[doneCourse.index(course), 'category'] = 'C1'
    elif course_id[0:2] == 'SO':
        df_doneCourse.loc[doneCourse.index(course), 'category'] = 'C2'
    elif course_id[0:2] == 'SC':
        df_doneCourse.loc[doneCourse.index(course), 'category'] = 'C3'
    else:
        if course[4] and course_id[:2] == student_aca_code:
            # 若為它系畢修，將視為選修學分
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'A1'
        else:
            df_doneCourse.loc[doneCourse.index(course), 'category'] = 'A2'

## DataFrame 轉 JSON

In [9]:
all_done_Course = [] #總課表

# 將各 series 轉為 dict 放置 all_course list
for i in range(len(df_doneCourse)):
    tempCourse = df_doneCourse.loc[i,].to_dict()
    all_done_Course.append(tempCourse)
# 存入 AllCourse_new.json
with open('StudentDoneCouse.json','w',encoding='utf8') as f:
    f.write(json.dumps(all_done_Course, ensure_ascii=False).encode("utf8",errors='ignore').decode("utf8",errors='ignore'))

In [10]:
df_doneCourse

,id,name,credit,score,category
0,AMA500,基礎微積分,2,73,A2
1,IFDA509,史丹佛設計思考與科技創新,2,93,A2
2,CSA001,計算機概論,2,90,A1
3,CSA011,離散數學,3,90,A1
4,CSA02A,微積分(一),3,87,A1
5,CSA07A,程式設計與實習（一）,2,97,A1
6,GR5403,中文(上),2,95,AC
7,GRA1B0,英語會話與閱讀(上)：中階班,2,88,AE
8,GRF109,服務學習培養(創新學院 A),0,95,A3
9,GRS55A,體育(一)-資工系,0,92,A3


## Session 技巧
* 儲存與伺服器的紀錄
* 連線不中斷

`req = requests.Session()`

`req.post('https://aca.nuk.edu.tw/Student2/Menu1.asp',{'Account':account,'Password':password})`操作

`r = req.post('https://aca.nuk.edu.tw/Student2/SO/ScoreQuery.asp',data={'Classno':''})`存取內容

[參考](https://blog.csdn.net/haeasringnar/article/details/82558729)